In [28]:
# ----------------------------- This cell be detect hand and will remove background from image.------------------
import argparse
import glob
import os
import cv2
import pathlib
from PIL import Image
from rembg.bg import remove
from yolo import YOLO
from pathlib import Path

dataset_folder = "signs"
resultfolder = "abc"

yolo = YOLO("models/cross-hands.cfg", "models/cross-hands.weights", ["hand"])
yolo.size = 640
yolo.confidence = 0.40

conf_sum = 0
detection_count = 0

data_dir = pathlib.Path(f'{dataset_folder}/')                 
file_paths = list(data_dir.glob('*/*.jpg'))
hand_with_bg = True

# files
for file in file_paths:

    path = Path(file)
    head, tail = os.path.split(path)
    path = os.path.join( os.path.split(head)[-1],  tail)
    path = os.path.join(resultfolder,  path)
    print(path)
    
    if not os.path.exists(path):
        file = os.path.join(file)
        mat = cv2.imread(file)
        mat = cv2.resize(mat, (256,256))
        width, height, inference_time, results = yolo.inference(mat)

        print("%s in %s seconds: %s classes found!" %
            (os.path.basename(file), round(inference_time, 2), len(results)))

        output = []
        for detection in results:
            id, name, confidence, x, y, w, h = detection
            cx = x + (w / 2)
            cy = y + (h / 2)

            conf_sum += confidence
            detection_count += 1

            # draw a bounding box rectangle and label on the image
            color = (255, 0, 255)
            segment = mat[y:y+h, x:x+w]
            
            if not os.path.exists(os.path.dirname(path)):
                try:
                    os.makedirs(os.path.dirname(path))
                except: # Guard against race condition
                    print("folder error")
            try:
                segment = cv2.resize(segment, (256,256))
                
                if hand_with_bg:
                    segment = cv2.cvtColor(segment,cv2.COLOR_BGR2GRAY)
                    cv2.imwrite(path, segment)
                else:
                    img = cv2.cvtColor(segment, cv2.COLOR_BGR2RGB)
                    im_pil = Image.fromarray(img)
                    cutout = remove(im_pil)
                    cutout.save(path, "PNG")
            except:
                print("Error")

In [ ]:
# --------------------------------- Use this code to detect skin only ----------------------------------
import argparse
import glob
import os
import cv2
import pathlib
from pathlib import Path
import skin_detector

dataset_folder = "result/signs"
resultfolder = "skin"

conf_sum = 0
detection_count = 0

data_dir = pathlib.Path(f'{dataset_folder}/')                 
file_paths = list(data_dir.glob('*/*.jpg'))

# files
for file in file_paths:
    
    path = Path(file)
    path = str(path).split("\\")
    path = path[-2]+"\\"+path[-1]
    path = os.path.join(resultfolder,  path)
    
    if not os.path.exists(path):
        file = os.path.join(file)
        print(path)
        img_col = cv2.imread(file, 1)
    
        segment = skin_detector.process(img_col)

        if not os.path.exists(os.path.dirname(path)):
            try:
                os.makedirs(os.path.dirname(path))
            except: # Guard against race condition
                print("folder error")
        try:
            segment = cv2.bitwise_and(img_col, img_col, mask=segment)
            cv2.imwrite(path, segment)
        except Exception as e:
            print(e)